# Kaggle 그랜드마스터 처럼 접근하기 (https://www.kaggle.com/code/jhoward/iterate-like-a-grandmaster)
처럼 접근하기 😎
1. Creating an effective validation set
2. Iterating rapidly to find changes which improve results on the validation set.

## References
- [Iterate like a grandmaster!](https://www.kaggle.com/code/jhoward/iterate-like-a-grandmaster) (source)
- [Cooperative Patent Classification Codes Meaning](https://www.kaggle.com/datasets/xhlulu/cpc-codes) (additional dataset)
- [deberta_v3_small offline](https://www.kaggle.com/datasets/jonathanchan/deberta-v3-small) (model package)
- [HuggingFace Datasets 1.11.0. Wheals for offline mode.](https://www.kaggle.com/datasets/oleksandrsirenko/huggingface-datasets) (lib package)

## 효율적인 GPU 리소스 관리 및 AI 를 위한 패키지 세트 활용

In [ ]:
# A lot of the basic imports you'll want (np, pd, plt, etc) are provided by fastai, so let's grab them in one line
from fastai.imports import *  

# It's nice to be able to run things locally too, to save your Kaggle GPU hours, so set a variable to make it easy to see where we are
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

## 데이터 세트

In [ ]:
path = (Path('../input/us-patent-phrase-to-phrase-matching') if iskaggle else Path.home()/'data'/'us-patent-phrase-to-phrase-matching')
path.ls()

In [ ]:
cpc_code_path = (Path('../input/cpc-codes') if iskaggle else Path.home()/'data'/'cpc-codes')
cpc_code_path.ls()

## 데이터 세트 분석 (들여다보기...)

In [ ]:
df = pd.read_csv(path/'train.csv')
df.head(5)

In [ ]:
eval_df = pd.read_csv(path/'test.csv')
eval_df.head(5)

In [ ]:
df.target.value_counts()

In [ ]:
df.anchor.value_counts()

In [ ]:
df.context.value_counts()

In [ ]:
df['section'] = df.context.str[0]
df.section.value_counts()

In [ ]:
eval_df['section'] = eval_df.context.str[0]
eval_df.section.value_counts()

In [ ]:
df.score.hist()

In [ ]:
df[df.score==1]

In [ ]:
df[df.score==1].section.value_counts()

In [ ]:
df[df.score==0.5]

In [ ]:
df[df.score==0.5].section.value_counts()

In [ ]:
cpc_code_df = pd.read_csv(cpc_code_path/'titles.csv')
cpc_code_df

In [ ]:
cpc_code_df.code.value_counts()

In [ ]:
cpc_code_df.section.value_counts()

In [ ]:
cpc_code_df[cpc_code_df.code == 'A']

In [ ]:
cpc_code_df[['code', 'title', 'section']]

In [ ]:
cpc_code_df[cpc_code_df.section == 'A']

In [ ]:
cpc_code_df[cpc_code_df.section == 'A'].title

In [ ]:
cpc_code_df[cpc_code_df.code == 'A47']

### 분석 결론
1. anchor 텍스트와 target 텍스트만 가지고 의미 유사성 수치를 재단하는 것은 힘들 것 (하지만 불가능 하진 않음)
2. 데이터 세트에는 anchor 와 target 에 대한 추가자료로 context 라는 분류 코드를 제공하는데 이에 대한 자료를 cpc-code 데이터 세트(public)에서 확인 가능
3. cpc-code 자료에서 context 분류 코드에 대한 description 을 얻을 수 있음
4. cpc-code 자료에서 context 의 section description 과 context description 을 확인할 수 있었고 둘 모두 사용하기

참고) A47(context) = A(section) + 47(subclass)

## 문제 해결을 위한 모델링 단계

### Huggingface Transformers 패키지 사용

In [ ]:
from torch.utils.data import DataLoader
import warnings,transformers,logging,torch
from transformers import TrainingArguments,Trainer
from transformers import AutoModelForSequenceClassification,AutoTokenizer

if iskaggle:
    !pip install --no-index --find-links ../input/huggingface-datasets datasets -qq
import datasets
from datasets import load_dataset, Dataset, DatasetDict

warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

In [ ]:
model_nm = '../input/deberta-v3-small/deberta-v3-small'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_nm)

In [ ]:
# check the special separator token
sep = tokenizer.sep_token
sep

### 소분류에 해당하는 context description 수집

In [ ]:
df_1 = df.merge(cpc_code_df[['code', 'title']], left_on='context', right_on='code', how='left')
df_1

In [ ]:
eval_df_1 = eval_df.merge(cpc_code_df[['code', 'title']], left_on='context', right_on='code', how='left')
eval_df_1.head(3)

### 대분류에 해당하는 section description 수집

In [ ]:
df_2 = df.merge(cpc_code_df[['code', 'title']], left_on='section', right_on='code', how='left')
df_2

In [ ]:
eval_df_2 = eval_df.merge(cpc_code_df[['code', 'title']], left_on='context', right_on='code', how='left')
eval_df_2.head(3)

### 입력 데이터로 사용할 문장 조립
input = (대분류 설명) + SEP + (소분류 설명) + SEP + (anchor 문장) + SEP + (target 문장)

In [ ]:
df['inputs'] = df_2.title + sep + df_1.title + sep + df.anchor + sep + df.target
df

In [ ]:
eval_df['inputs'] = eval_df_2.title + sep + eval_df_1.title + sep + eval_df.anchor + sep + eval_df.target
eval_df.head(3)

### huggingface Datasets 를 사용, pandas 데이터프레임을 Dataset 으로 변환

In [ ]:
ds = Dataset.from_pandas(df).rename_column('score', 'label')
eval_ds = Dataset.from_pandas(eval_df)

In [ ]:
# Dataset.map 에서 사용하기 위한 토크나이징 호출 함수 생성
def tokenizing_func(x):
    return tokenizer(x['inputs'])

tokenizing_func(ds[0])  # test

In [ ]:
tok_ds = ds.map(tokenizing_func, batched=True, remove_columns=('anchor','target','context','inputs','id','section'))

In [ ]:
tok_eval_ds = eval_ds.map(tokenizing_func, batched=True, remove_columns=('anchor','target','context','inputs','id','section'))

### 검증용 데이터 세트 만들기
원문에 작성된 내용에 따르면, 해당 [링크](https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/discussion/315220)내 자료에 따르면 실제 평가용 샘플에는 학습 세트와 완전히 다른 anchor 를 사용한다는 것. 보다 효과적인 검증용 데이터 세트를 만들기 위해 실제 평가용 세트와 유사하게 생성. 

In [ ]:
anchors = df.anchor.unique()  # 학습 세트 내 anchor 
np.random.seed(42)
np.random.shuffle(anchors)
val_prop = 0.25
val_sz = int(len(anchors)*val_prop)
val_anchors = anchors[:val_sz]
val_anchors

### pandas 와 numpy 를 적절히 섞어서 사용하면 검증용 샘플을 편리하게 구성 가능

In [ ]:
is_val = np.isin(df.anchor, val_anchors)  # 검증용 anchor 가 포함된 인덱스 체크
idxs = np.arange(len(df))
val_idxs = idxs[ is_val]
trn_idxs = idxs[~is_val]
len(val_idxs),len(trn_idxs)

### huggingface Dataset 로 numpy 나 pandas 와 비슷한 스타일을 지원함

In [ ]:
dds = DatasetDict({"train":tok_ds.select(trn_idxs), "test": tok_ds.select(val_idxs)})

In [ ]:
df.iloc[trn_idxs]

In [ ]:
df.iloc[val_idxs]

In [ ]:
df.iloc[trn_idxs].score.mean(),df.iloc[val_idxs].score.mean()  # 학습용 label 점수와 검증용 label 점수 분포 차이가 거의 없음을 보여줌

### 모델 셋업

In [ ]:
# transformers 패키지에서도 사용하기 위한 평가 함수 작성
def corr(eval_pred): return {'pearson': np.corrcoef(*eval_pred)[0][1]}

In [ ]:
# hyper-parameters
learning_rate = 8e-5
batch_size = 128
weight_decay = 0.01
epochs = 4

### Huggingface Trainer 와 이를 위한 TrainingArgument 를 사용 ([공식 레퍼런스](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments))
- Tensorflow & Pytorch 를 wrapping 한 상태로 학습 프로세스를 지원 (근데 메인은 pytorch)
- 복잡한 학습 테크닉을 알아야 하는 수고로움과 코드로 구현하는데 드는 노력을 최소화해주고 구현상 일어날 수 있는 실수들을 미리 방지할 수 있음
- 이러한 패키지를 사용하는 가장 큰 이유는 Machine Learning Workflow 마지막 단계인 optimization process 에 대한 실험 단계까지 쉽게 도달하기 위함임

In [ ]:
args = TrainingArguments('outputs', # output directory
                         learning_rate=learning_rate, 
                         warmup_ratio=0.1, 
                         lr_scheduler_type='cosine', 
                         fp16=True, 
                         evaluation_strategy="epoch", 
                         per_device_train_batch_size=batch_size, 
                         per_device_eval_batch_size=batch_size*2,
                         num_train_epochs=epochs,
                         weight_decay=weight_decay, 
                         report_to='none')

model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, 
                  args, 
                  train_dataset=dds['train'], 
                  eval_dataset=dds['test'],
                  tokenizer=tokenizer, 
                  compute_metrics=corr)

In [ ]:
# trainer.train()

### 초기 모델링 결과
- 0.8 정도의 준수한 성능을 검증 세트에서 얻을 수 있었음
- 위의 결과를 베이스라인으로 지정하고 개선점을 탐색

## Example) [원본 글](https://www.kaggle.com/code/jhoward/iterate-like-a-grandmaster)에서 소개하는 개선 방법 - SEP 토큰 제거

In [ ]:
# new sep token
sep = " [s] "

# input sentences
df['inputs'] = df_2.title + sep + df_1.title + sep + df.anchor + sep + df.target
eval_df['inputs'] = eval_df_2.title + sep + eval_df_1.title + sep + eval_df.anchor + sep + eval_df.target

# dataset
ds = Dataset.from_pandas(df).rename_column('score', 'label')
eval_ds = Dataset.from_pandas(eval_df)
tok_ds = ds.map(tokenizing_func, batched=True, remove_columns=('anchor','target','context','inputs','id','section'))
tok_eval_ds = eval_ds.map(tokenizing_func, batched=True, remove_columns=('anchor','target','context','inputs','id','section'))

# dataset dict
dds = DatasetDict({"train":tok_ds.select(trn_idxs), "test": tok_ds.select(val_idxs)})

# model
args = TrainingArguments('outputs', # output directory
                         learning_rate=learning_rate, 
                         warmup_ratio=0.1, 
                         lr_scheduler_type='cosine', 
                         fp16=True, 
                         evaluation_strategy="epoch", 
                         per_device_train_batch_size=batch_size, 
                         per_device_eval_batch_size=batch_size*2,
                         num_train_epochs=epochs,
                         weight_decay=weight_decay, 
                         report_to='none')

model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, 
                  args, 
                  train_dataset=dds['train'], 
                  eval_dataset=dds['test'],
                  tokenizer=tokenizer, 
                  compute_metrics=corr)

In [ ]:
trainer.train()

# Submission

In [ ]:
predictions = trainer.predict(tok_eval_ds).predictions
predictions = predictions.reshape((-1,))
predictions = [min(1.0, max(0., _)) for _ in predictions]
eval_df['score'] = predictions
eval_df[['id', 'score']]

In [ ]:
eval_df[['id', 'score']].to_csv('submission.csv', index=False)